# 0.0 Imports

In [4]:
import pandas as pd

## 0.1 Helper functions

## 0.2 Loading data

In [26]:
df_raw_1 = pd.read_csv('../data/train.csv', low_memory=False)
df_raw_2 = pd.read_csv('../data/store.csv', low_memory=False)

In [24]:
df_raw_1

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


# 1.0 Data description

## 1.1 Rename columns

## 1.2 Data dimensions

## 1.3 Data types

## 1.4 Check NA

## 1.5 Fillout NA 

## 1.6 Change Types

## 1.7 Descriptive Statistical

### 1.7.1 Numerical attributes

### 1.7.2 Categorical attributes

# 2.0 Feature engineering

## 2.1 Mapa mental de Hipoteses

## 2.2 Criação de Hipoteses

## 2.3 Lista final de Hipoteses

## 2.4 Feature engineering

# 3.0 Feature filtering

## 3.1 Filtragem das linhas

## 3.2 Seleção das colunas

# 4.0 EDA - Análise exploratória dos dados

## 4.1 Análise univariada

### 4.1.1 Response variable

### 4.1.2 Numerical variable

### 4.1.3 Categorical variable

## 4.2 Análise bivariada

### 4.2.x Hipothesis validation

### 4.2.y Hipothesis resume

## 4.3 Análise multivariada

### 4.3.1 Numerical attributes

### 4.3.2 Categorical attributes

# 5.0 Data Preparation

## 5.1 Normalization

## 5.2 Rescaling

## 5.3 Transformation

### 5.3.1 Encoding

### 5.3.2 Response variable transformation

### 5.3.3 Cyclical features encoding

# 6.0 Feature selection

## 6.1 Splitting dataframe

## 6.2 Boruta as Feature Selector

## 6.3 Manual feature selection

# 7.0 Machine Learning Modelling

## 7.1 Baseline model

## 7.2 Linear regression model

### 7.2.1 Linear regression model - Cross validation

## 7.3 Random Forest 

### 7.4.1 Random Forest  - Cross validation

## 7.5 XGBoost Regrssor

### 7.5.1 XGBoost Regressor - Cross Validation

## 7.6 Compare Model's Performance


### 7.6.1 Single performance

### 7.6.2 Real performance - Cross validation

# 8.0 Hyperparameter Fine Tuning

## 8.1 Random search

## 8.2 - Final model